# Required Libraries

There are quite a few libraries and toolkits in Python that provide implementations of various algorithms that you can use to build a recommender.

For this particular exercise, I am going to use [Surprise](https://github.com/NicolasHug/Surprise).Surprise is a Python SciKit that comes with various recommender algorithms and similarity metrics to make it easy to build and analyze recommenders.

**Note:** If for any reason you prefer to use another library that you are more familiar with or you want to explore, feel free!

Here’s how to install it using pip (more info about the installation on the [Surprise documentation](http://surpriselib.com/)):

In [ ]:
! pip install numpy # Numpy is required by surprise
! pip install scikit-surprise

# MovieLens 100K

We’ll be working with the well known [MovieLens 100K Dataset](https://grouplens.org/datasets/movielens/). This dataset is the smaller version of the MovieLense movie recommendation dataset, a de-facto standard for the evaluation of recommendation systems.

The dataset includes 100K user ratings about a collection of movies. While this is not the most advanced, neither the most realistic recommendation dataset, it has been masively used for experimentation, what gives you the opportunity to compare your system with the literature of recommender systems.

Additionally, this is a nice starting point to familiarize with the recommendation problem.

# Read Data

Please, download the dataset from the following [link](http://files.grouplens.org/datasets/movielens/ml-100k.zip) and unzip it.

The dataset provides different files (you can check the README for more information), but we are only interested in the u.data (ratings) and the u.item (description of the movies for a more readable list of results).

In [1]:
import pandas as pd

# Please ensure to update the filepath to where you have unizipped it.
ratings = pd.read_csv('ml-100k/u.data', sep='\t', error_bad_lines=False, encoding="latin-1", header=None)
ratings.columns = ['userID', 'itemID', 'rating', 'timestamp']

# Please ensure to update the filepath to where you have unizipped it.
movies = pd.read_csv('ml-100k/u.item', sep='|', error_bad_lines=False, encoding="latin-1", header=None)[[0,1]]
movies.columns = ['itemID', 'title']

# Merge both datasets together to facilitate their use
df = pd.merge(ratings, movies, on='itemID', how='inner')[['userID','title','rating']]

Let's quickly visualize the dataset to ensure that everything looks good

In [2]:
print('Dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::20000, :])

Dataset shape: (100000, 3)
-Dataset examples-
       userID                         title  rating
0         196                  Kolya (1996)       3
20000     610  2001: A Space Odyssey (1968)       3
40000     592                Kingpin (1996)       5
60000     435               Supercop (1992)       3
80000      87         Batman & Robin (1997)       4


# Analyze the data

Before starting to work with the data I would like to take a quick view at it. I strongly encourage you to always perform this step to be sure that the data that you have makes sense, that you do not have any format problem or error you need to correct and to perform a first analysis to understand if the data you have will be enought to create a recommender system (or to go for a simpler approach).

This step is highly speculative and it's mainly based on your expertise, but there are some things in which we can focus

### Ratings distribution

The first thing is to analyze the distribution of ratings to understand among other things:

- The rating scale: is it 0-5, 1-5, 1-10....
- The distribution of the ratings: are the users biased to high or low rates?
- Are the users using the entire rating scale? e.g.: sometimes people performs binary ratings even if you provide them with a more granular scale (5 good, 1 bad and nothing in between)

**Note:** Don't be scared by the following code. Basically, I am counting the number of times each rating occurs in the dataset and plot the data in a fancy plotly histogram that you can interact with. If you are not very well versed in Python, you can just perform the counting an print the results

In [3]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Count the number of times each rating appears in the dataset
data = df['rating'].value_counts().sort_index(ascending=False)

# Create the histogram
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} movie ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

We can see that ratings are biased to positive score. This tends to be common in many recommendation scenarios (for example in Google Reviews a restaurant with a scoring smaller than 4/3.5 if definitely a no-go). That's because we have the bias terms in the recommendation algorithms (later on we will see them in practice)

### Numbers of ratings per movie

The following analysis checks how many ratings (interactions) each movie has. As explained in class, due to the long tail "problem", we will have a tiny amount of highly popular movies (with many ratings) and a very large number of almost unknown movies (with a handful of ratings).

The degree in which this long tail phenomena occurs will be related to aspects such as serendipity or overespecialization of the recommendations and the recommendation methodology that you should select (i.e., if you only have ratings for a bunch of movies, perhaps it does not make sense to use a CF recommendation system)

In [4]:
# Number of ratings per movie
data = df.groupby('title')['rating'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Movie',
                   xaxis = dict(title = 'Number of Ratings Per Movie'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Most of the movies received less than 10 ratings, and very few have many ratings. As discussed in class, this is very typical (and in fact one of the main problems) of Recommender Systems.

**Note:** This effect is even smaller in the MovieLens dataset since it is an experimental dataset that has been curated for it (i.e., from the documentation: "Each user has rated at least 20 movies").


### Number of ratings per user

Now it's time to see how many ratings we have per user. As previously noted, because of how the dataset was created, the minimum number of ratings per user in this dataset is 20, which alleviates the long-tail problem (i.e., when most of your users has very few or 0 ratings). 

Since this is an experimental dataset for research, they have created it this way. However, do not expect that in the real world. In fact, a common pre-processing step in recommendation is to remove those users/items for which you do not have many ratings and focus on the rest (at least for creating your model).

In [5]:
# Number of ratings per user
data = df.groupby('userID')['rating'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0, size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

As expected similar distribution

From this analysis, I do not see anything weird. I wasn't expected to see anything since, again, this is an experimental dataset, properly curated to avoid you any of the real-world problems. Nevertheless, it is always worthy to check, just in case. 

# Managing data with Surprise

It's now time to convert our datasets to the format required by the Surprise library. This step is pretty straightforward since the library provides some handy methods to do it (check them in the library documentation in case you want to know more).

To load a data set from the above pandas data frame, we will use the *load_from_df()* method, we will also need a Reader object, and the rating_scale parameter must be specified. The data frame must have three columns, corresponding to the user ids, the item ids, and the ratings in this order. Each row thus corresponds to a given rating.

In [6]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userID', 'title', 'rating']], reader)

# First Try: Neighbourhood-based Collaborative Filtering

We are going to start by using one of the simplest recommendation methodologies (Neighbourhood-based CF). This technique is pretty simple and fast, but it provides accurate results for many scenarios.

There are several Neighbourhood-based CF implementations in Surprise: https://surprise.readthedocs.io/en/stable/knn_inspired.html?highlight=neighbor. The choice of algorithm for the recommender function depends on the technique you want to use. For this first experiment, we are going to use the Baseline approach

To specify the parameters of the execution, you simply have to configure the function by passing a dictionary as an argument to the recommender function. The dictionary should have the required keys, such as the following:

- **name** contains the similarity metric to use. Options are cosine, msd, pearson, or pearson_baseline. The default is msd.
- **user_based** is a boolean that tells whether the approach will be user-based or item-based. The default is True, which means the user-based approach will be used.
- **min_support** is the minimum number of common items needed between users to consider them for similarity. For the item-based approach, this corresponds to the minimum number of common users for two items.
    
In particular, I will use the cosine distance as similarity metric for finding the neighbours, using the item-based approach

In [7]:
from surprise import KNNBaseline

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
knn = KNNBaseline(sim_options=sim_options)

Now that we have defined the algorithm configuration, we can actually test it. Again, Surprise provides some handy functions for it, in particular it includes a function to evaluate the algorithm via Cross Validation

In [8]:
from surprise.model_selection import cross_validate

results = cross_validate(knn, data, measures=['RMSE'], cv=3, verbose=True)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9460  0.9447  0.9509  0.9472  0.0027  
Fit time          2.28    1.99    2.65    2.31    0.27    
Test time         7.60    6.76    7.16    7.17    0.34    


The RMSE is about 0.95 (i.e., the average error between the predicted and the actual ratings). 

In order to actually understand the value of these results, my recommendation is to always comapare them to a very simple baseline approach implementing whatever logic or business rule you consider valid. For example: randomly recommend from the 10 most common books. 

The Surprise library implements different baseline methodologies (https://surprise.readthedocs.io/en/stable/basic_algorithms.html#). I am going to use the *NormalPredictor* recommendation that predicts a random rating based on the distribution of the training set, which is assumed to be normal.

In [9]:
from surprise import NormalPredictor

cross_validate(NormalPredictor(), data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5188  1.5243  1.5269  1.5234  0.0034  
Fit time          0.11    0.12    0.12    0.12    0.01    
Test time         0.25    0.23    0.26    0.25    0.01    


{'test_rmse': array([1.51877873, 1.52434677, 1.52694327]),
 'fit_time': (0.10603952407836914, 0.11796331405639648, 0.12400484085083008),
 'test_time': (0.25350189208984375, 0.2265002727508545, 0.2579948902130127)}

RMSE is larger (as expected) what tells us that:
- KNN is actually learning from the training data
- 1.5 is the lower threshold for any recommender system in this dataset

## Tuning the Algorithm Parameters

As we explained, you can provide a dictionary with the desired configuration (i.e., similarity metric, user vs item based,...) to your recommendation algorithm. Ideally, you would like to test different configurations and select the one that best performs for your dataset and recommendation problem.

To that end, Surprise provides a GridSearchCV class analogous to GridSearchCV from scikit-learn.
With a dict of all parameters, GridSearchCV tries all the combinations of parameters and reports the best parameters for any accuracy measure

For example, you can check which similarity metric works best for your data in memory-based approaches:

In [10]:
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

The best configuration corresponds to a item-based configuration using Mean Square distance with a min support equals to 3, which is able to slightly reduce the RMSE to 0.942.


# Second Try: Matrix Factorization

Once we have understood the recommendation problem and how the Surprise library works, it's time to move to more interesting approaches, Matrix Factorization in particular.

Surprise provides different implementations of well-known matrix factorization methodologies: https://surprise.readthedocs.io/en/stable/matrix_factorization.html

**Please, review the different matrix factorization approaches and use the one that you prefer (you can test and compare several ones if you want). You can also test different parameter configurations (as I did in the exercise above) to see if you can optimize the results**

In [11]:
# Your code here

**Analyze the achieved results. Are they better that those offered by the Neighbourhood-based approach? By how much? Do you think that it makes sense to apply SVD given these results? Why or Why not?**

# Third Try: Benchmarking

**If you have been able to complete the previous exercise, you can try to test other algorithms beyond Matrix Factorization and compare their results**


In [1]:
# Your code here 

**Analyze the results**